In [10]:
import geemap.geemap as map
import ee



In [12]:
ee.Authenticate()
ee.Initialize(project='ee-rohitshinde090103')

In [15]:
m=map.Map()
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [17]:
Goa = ee.FeatureCollection('projects/rohitshinde4680/assets/goa')
m.addLayer(Goa,{},'goa')
m

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [18]:
Goa_ESA = ee.ImageCollection("ESA/WorldCover/v200").first()

In [20]:
Goa_clipped = Goa_ESA.clip(Goa)

In [21]:
GoaVis = {'bands':['Map']}

In [22]:
m.addLayer(Goa_clipped,GoaVis,'Goa LULC')
m

Map(bottom=119956.0, center=[15.485445179478619, 74.05677795410158], controls=(WidgetControl(options=['positio…

In [23]:
Legend = {
    'Tree cover':'#006400',
    'Shrubland':'#ffbb22',
    'Grassland':'#ffff4c',
    'Cropland':'#f096ff',
    'Built-up':'#fa0000',
    'Bare/sparse vegetation':'#b4b4b4',
    'Snowandice':'#f0f0f0',
    'Permanent water bodies':'#0064c8',
    'Herbaceous wetland':'#0096a0',
    'Mangroves':'#00cf75',
    'Moss and lichen':'#fae6a0'
}

In [24]:
m.add_legend(title='ESA Land Cover classes',legend_dict= Legend)
m

Map(bottom=60267.0, center=[15.117204423332618, 75.01739696840995], controls=(WidgetControl(options=['position…

In [25]:
landsat8_Data = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterDate('2014-03-01','2014-05-31').filterMetadata('CLOUD_COVER','less_than',10).mean().clip(Goa)



In [26]:
m.addLayer(landsat8_Data,{},'Landsat 8_2014')
m

Map(bottom=60267.0, center=[15.117204423332618, 75.01739696840995], controls=(WidgetControl(options=['position…

In [27]:
TP = Goa_clipped.sample(region=Goa.geometry(),scale=20,numPixels=1500,seed=1,geometries=True)

In [28]:
m.addLayer(TP,{},'Training Points')
m

Map(bottom=60267.0, center=[15.117204423332618, 75.01739696840995], controls=(WidgetControl(options=['position…

In [29]:
bands = ['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7']
label = 'Map'

In [30]:
training = landsat8_Data.select(bands).sampleRegions(**{
    'collection':TP,
    'properties':[label],
    'scale':30
})

In [31]:
classfiertrained =ee.Classifier.smileCart().train(training,label,bands)

In [32]:
result =landsat8_Data.select(bands).classify(classfiertrained)

In [33]:
m.addLayer(result,{},'Supervised Classification')
m

Map(bottom=60267.0, center=[15.117204423332618, 75.01739696840995], controls=(WidgetControl(options=['position…